In [1]:
import requests
import pandas as pd
import time
from datetime import datetime

In [2]:
palavras_chave = [
    'dados abertos',
    'dados publicos'
]

[Construindo uma consulta](https://developer.github.com/v3/search/#constructing-a-search-query)

`q=SEARCH_KEYWORD_1+SEARCH_KEYWORD_N+QUALIFIER_1+QUALIFIER_N`

### Consultas conhecidas
- Consulta para repositórios com mais uma 1 star
`q = 'q=stars:>1'`

In [3]:
q = 'q=dados+abertos'
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?'
url_final = url_base+q+sort
credentials=('lorenaps','')
url_final

'https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc'

Verificando o limite de requisições

In [4]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1568660876},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1568657336},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1568660876},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1568660876}},
 'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1568660876}}

In [5]:
colunas=[
        'id',
        'full_name',
        'description',
        'owner_type', 
        'owner_url',
        'owner_html_url',
        'html_url',
        'url',
        'fork',
        'created_at',
        'updated_at',
        'size',
        'stargazers_count',
        'language',
        'has_issues',
        'has_wiki',
        'forks_count',
        'forks',
        'open_issues',
        'watchers',
        'timestamp_extract',
        'Readme'
    ]


In [6]:
str(time.time()).split('.')[0] # Pegar timestamp atual

'1568657283'

In [7]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('full_name', None),
                        item.get('description', None),      
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=colunas)

    return df    

In [8]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

In [9]:
def get_total_paginas(data):
    
    itens_por_pagina = len(data.get('items'))
    total_paginas = data.get('total_count') // itens_por_pagina # opreração de div em python
    total_paginas

    print('Total de registros:{0} , Registros por página:{1}, Total de Páginas:{2}'.format(
        data.get('total_count'), itens_por_pagina, total_paginas))
    
    return total_paginas        

In [10]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://developer.github.com/v3/search/'}

In [16]:
def percorrendo_paginas(url):
    
    resultados = pd.DataFrame(columns=['id',
                                        'full_name',
                                        'description',
                                        'owner_type', 
                                        'owner_url',
                                        'owner_html_url',
                                        'html_url',
                                        'url',
                                        'fork',
                                        'created_at',
                                        'updated_at',
                                        'size',
                                        'stargazers_count',
                                        'language',
                                        'has_issues',
                                        'has_wiki',
                                        'forks_count',
                                        'forks',
                                        'open_issues',
                                        'watchers',
                                        'timestamp_extract',
                                        'commits',
                                        'contributors',])

    print('Extraindo página:1')
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    resultados = extrair_dados(data, resultados)
    
    iteracoes = 34 
    
    for iteracao in range(1, 34):
        print("\n>>>>>>>> Iteracao:{0}".format(iteracao+1))
        print("Tempo atual:{0}".format(datetime.now()))

        # Para requisições não autenticadas a api restringe para 10 requisições por minuto,
        # Para requisições autenticadas 30 por minuto.
        #if iteracao % 10 == 0:
        #    print("sleep 1 minuto")
        #    time.sleep(60)
        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            print("Next url extraída: {0}".format(next_url))

            print('Extraindo página:{0}'.format(iteracao+1))
            results = requests.get(next_url, auth=credentials)
            print('Status:{0}'.format(results))

            data = dict(results.json())

            resultados = extrair_dados(data, resultados)
        
    return resultados

In [17]:
%%time
resultados = percorrendo_paginas(url_final)

Extraindo página:1

>>>>>>>> Iteracao:2
Tempo atual:2019-09-16 15:11:53.440383
Next url extraída: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=2
Extraindo página:2
Status:<Response [200]>

>>>>>>>> Iteracao:3
Tempo atual:2019-09-16 15:11:54.993983
Next url extraída: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=3
Extraindo página:3
Status:<Response [200]>

>>>>>>>> Iteracao:4
Tempo atual:2019-09-16 15:11:56.487518
Next url extraída: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=4
Extraindo página:4
Status:<Response [200]>

>>>>>>>> Iteracao:5
Tempo atual:2019-09-16 15:11:57.952272
Next url extraída: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=5
Extraindo página:5
Status:<Response [200]>

>>>>>>>> Iteracao:6
Tempo atual:2019-09-16 15:11:59.505109
Next url extraída: https://api.github.com/search/repositories?q=dados+abertos&sor

In [18]:
resultados.describe()

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
count,355,355,324,355,355,355,355,355,355,355,...,263,355,355,355,355,355,355,355,0.0,0.0
unique,355,355,322,2,302,302,355,355,1,355,...,20,2,2,14,14,13,25,12,0.0,0.0
top,125292029,gcianfarani/jogo-dados-abertos,Sistema para o 2º Concurso de aplicativos para...,User,https://api.github.com/users/febr-team,https://github.com/febr-team,https://github.com/SabrinyBezerra/Dashboard,https://api.github.com/repos/WelitonAmartins/C...,False,2012-12-21T02:11:01Z,...,Python,True,True,0,0,0,0,1568657521,NaN,NaN
freq,1,1,2,280,10,10,1,1,355,1,...,54,347,347,271,271,307,237,30,NaN,NaN


In [19]:
resultados.head()

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,None,True,True,7,7,201,132,1568657513,NaN,NaN
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,Python,True,True,32,32,1,99,1568657513,NaN,NaN
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,Jupyter Notebook,True,True,17,17,0,48,1568657513,NaN,NaN
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,None,True,True,8,8,1,48,1568657513,NaN,NaN
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,CSS,True,False,15,15,10,38,1568657513,NaN,NaN


In [21]:
resultados.to_csv('../dados/repositorios.csv', index=False)

In [22]:
resultados.columns.values

array(['id', 'full_name', 'description', 'owner_type', 'owner_url',
       'owner_html_url', 'html_url', 'url', 'fork', 'created_at',
       'updated_at', 'size', 'stargazers_count', 'language', 'has_issues',
       'has_wiki', 'forks_count', 'forks', 'open_issues', 'watchers',
       'timestamp_extract', 'commits', 'contributors'], dtype=object)

## Extraindo commits e contributors através da biblioteca PyGithub

Links:
- https://github.com/PyGithub/PyGithub

In [24]:
from github import Github
from github import Repository
from github import ContentFile
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [25]:
import sys 
options = webdriver.ChromeOptions()
options.add_argument('headless')

In [26]:
token = ''

In [27]:
g = Github(token)

In [38]:
resultados_update = pd.read_csv('../dados/repositorios_update.csv')
resultados_update.head()

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,NaN,True,True,7,7,201,132,1568657513,7.0,NaN
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,Python,True,True,32,32,1,99,1568657513,38.0,3.0
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,Jupyter Notebook,True,True,17,17,0,48,1568657513,18.0,NaN
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,NaN,True,True,8,8,1,48,1568657513,40.0,NaN
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,CSS,True,False,15,15,10,38,1568657513,361.0,6.0


In [39]:
resultados_update.isna().sum()

id                     0
full_name              0
description           31
owner_type             0
owner_url              0
owner_html_url         0
html_url               0
url                    0
fork                   0
created_at             0
updated_at             0
size                   0
stargazers_count       0
language              92
has_issues             0
has_wiki               0
forks_count            0
forks                  0
open_issues            0
watchers               0
timestamp_extract      0
commits              221
contributors         345
dtype: int64

In [30]:
def get_contributors(url_repo):
    page = requests.get(url_repo)
    page_parse = bs(page.text, 'html.parser')
    lista = page_parse.findAll('span', attrs={'class': 'num text-emphasized'}) 
    
    print("#################################")
    print(lista)
    print("#################################")

    text = lista[3].text # Índice utilizado para pegar a posição correta da informação sobre os contributors no html
    contributors = text.replace(" ", "").replace("\n", "").replace(",", "")
    #contributors = int(contributors)
    
    return(contributors)

In [31]:
def get_contributors_with_seleium(url_repo):
    browser = webdriver.Chrome(options=options)
    browser.get(url_repo)
    page_parse = bs(browser.page_source, 'html.parser')
    lista = page_parse.findAll('span', attrs={'class': 'num text-emphasized'}) 
    
    print("#################################")
    print(lista)
    print("#################################")

    text = lista[3].text # Índice utilizado para pegar a posição correta da informação sobre os contributors no html
    contributors = text.replace(" ", "").replace("\n", "").replace(",", "")
    #contributors = int(contributors)
    
    return(contributors)

In [40]:
id_repos = resultados_update.loc[resultados_update['commits'].isnull()]['id']
len(id_repos)

221

In [42]:
resultados_update.loc[resultados_update['full_name'] == 'Api-DadosAbertos-Eventos']

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors


In [44]:
it = 0
total = len(id_repos)

for id_repo in id_repos[1:]:
    
    it = it + 1
    print("\nIteração:{0}/{1}".format(it, total))
    
    repo = g.get_repo(id_repo)
    print("Nome do repositório:{0}".format(repo.name))

    commits = repo.get_commits().totalCount
    print("Commits: ", commits)

    url_repo = resultados_update.loc[resultados_update["id"] == id_repo]['html_url'].to_list()[0]
    print("Url: ", url_repo)

    print("Atualizando data frame :)")
    resultados_update.loc[resultados_update["id"] == id_repo, 'commits'] = commits 

    resultados_update.to_csv('../dados/repositorios_update.csv', index=False)


Iteração:1/221
Nome do repositório:Camara_DadosAbertos
Commits:  2
Url:  https://github.com/victorsfleite/Camara_DadosAbertos
Atualizando data frame :)

Iteração:2/221
Nome do repositório:pdi-DadosAbertos
Commits:  5
Url:  https://github.com/darkbebs/pdi-DadosAbertos
Atualizando data frame :)

Iteração:3/221
Nome do repositório:DadosAbertosBrasil
Commits:  15
Url:  https://github.com/vfgoncalves/DadosAbertosBrasil
Atualizando data frame :)

Iteração:4/221
Nome do repositório:dadosBrasil
Commits:  9
Url:  https://github.com/luanoliveira1992/dadosBrasil
Atualizando data frame :)

Iteração:5/221
Nome do repositório:DadosAbertosDeputados
Commits:  9
Url:  https://github.com/karenazevedo/DadosAbertosDeputados
Atualizando data frame :)

Iteração:6/221
Nome do repositório:open-data-BR
Commits:  34
Url:  https://github.com/renvmorales/open-data-BR
Atualizando data frame :)

Iteração:7/221
Nome do repositório:observatorio-niteroi
Commits:  22
Url:  https://github.com/bcbernardo/observatorio-ni

GithubException: 409 {'message': 'Git Repository is empty.', 'documentation_url': 'https://developer.github.com/v3/repos/commits/#list-commits-on-a-repository'}

In [45]:
resultados_update = pd.read_csv('../dados/repositorios_update.csv')
id_repos = resultados_update.loc[resultados_update['contributors'].isnull()]['id']
len(id_repos)

345

In [46]:
it = 0
total = len(id_repos)

for id_repo in id_repos[1:]:
    it = it + 1
    print("\nIteração:{0}/{1}".format(it, total))
    
    url_repo = resultados_update.loc[resultados_update["id"] == id_repo]['html_url'].to_list()[0]
    print("Url: ", url_repo)

    contributors = get_contributors_with_seleium(url_repo)
    print("Contributors: ", contributors)

    print("Atualizando data frame :)")
    resultados_update.loc[resultados_update["id"] == id_repo, 'contributors'] = contributors

    resultados_update.to_csv('../dados/repositorios_update.csv', index=False)


Iteração:1/345
Url:  https://github.com/prefeiturasp/dados-educacao
#################################
[<span class="num text-emphasized">
              18
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:2/345
Url:  https://github.com/dadosgovbr/aplicativos-dados-brasil
#################################
[<span class="num text-emphasized">
              40
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:3/345
Url:  https://github.com/mapaslivres/localidades
#################################
[<span class="num text-emphas

#################################
[<span class="num text-emphasized">
              7
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:21/345
Url:  https://github.com/danielmbicalho/Dados_repositorios_e_ferramentas
#################################
[<span class="num text-emphasized">
              7
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:22/345
Url:  https://github.com/luisleao/oicamara
#################################
[<span class="num text-emphasized">
              27
            </span>, <span class="num t

#################################
[<span class="num text-emphasized">
              11
            </span>, <span class="num text-emphasized">
            2
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized">
      1
    </span>]
#################################
Contributors:  1
Atualizando data frame :)

Iteração:40/345
Url:  https://github.com/geovannyAvelar/Dados-Abertos-Campeonato-Brasileiro
#################################
[<span class="num text-emphasized">
              9
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:41/345
Url:  https://github.com/nymarya/prf_api
#################################
[<span class="num text-emphasized">
              55
            </span>, <s

#################################
[<span class="num text-emphasized">
              1
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:59/345
Url:  https://github.com/anp-qualidade/dados-abertos-pmqc
#################################
[<span class="num text-emphasized">
              86
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized">
      2
    </span>]
#################################
Contributors:  2
Atualizando data frame :)

Iteração:60/345
Url:  https://github.com/sergiohampel/hoteis-recife
#################################
[<span class="num text-emphasized">
              9
            </span>, <span clas

#################################
[<span class="num text-emphasized">
              25
            </span>, <span class="num text-emphasized">
            2
          </span>, <span class="num text-emphasized">
            3
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:78/345
Url:  https://github.com/febr-team/febr-data
#################################
[<span class="num text-emphasized">
              52
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:79/345
Url:  https://github.com/brunousml/Maquiavel
#################################
[<span class="num text-emphasized">
              7
            </span>, <span class="num text-emphasized">
        

#################################
[<span class="num text-emphasized">
              6
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
              0
            </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  0
Atualizando data frame :)

Iteração:97/345
Url:  https://github.com/pr-snas/nosmapas
#################################
[<span class="num text-emphasized">
              18
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:98/345
Url:  https://github.com/febr-team/febr-validacao
#################################
[<span class="num text-emphasized">
      

#################################
[<span class="num text-emphasized">
              1
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:116/345
Url:  https://github.com/EvaCosta/DadosAbertos
#################################
[<span class="num text-emphasized">
              3
            </span>, <span class="num text-emphasized">
            1
          </span>, <span class="num text-emphasized">
            0
          </span>, <span class="num text-emphasized"></span>]
#################################
Contributors:  
Atualizando data frame :)

Iteração:117/345
Url:  https://github.com/renato-bohler/mapa-dados-abertos
#################################
[<span class="num text-emphasized">
              43
            </span>, <span class="num text-empha

IndexError: list index out of range

In [36]:
resultados_update.head()

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,NaN,True,True,7,7,201,132,1568657513,7.0,
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,Python,True,True,32,32,1,99,1568657513,38.0,3
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,Jupyter Notebook,True,True,17,17,0,48,1568657513,18.0,
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,NaN,True,True,8,8,1,48,1568657513,40.0,
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,CSS,True,False,15,15,10,38,1568657513,361.0,6


# Verificando dados extraídos

In [37]:
resultados_update.isna().sum()

id                     0
full_name              0
description           31
owner_type             0
owner_url              0
owner_html_url         0
html_url               0
url                    0
fork                   0
created_at             0
updated_at             0
size                   0
stargazers_count       0
language              92
has_issues             0
has_wiki               0
forks_count            0
forks                  0
open_issues            0
watchers               0
timestamp_extract      0
commits              221
contributors         323
dtype: int64

### Repositórios onde as linguaguagens vem como Nan
Aparentemente são repostirórios de apostilas, listas, textos.

In [175]:
resultados_update.loc[resultados_csv['language'].isnull()]

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
6,13491895,Organization,https://api.github.com/users/EbookFoundation,https://github.com/EbookFoundation,https://github.com/EbookFoundation/free-progra...,https://api.github.com/repos/EbookFoundation/f...,False,2013-10-11T06:50:37Z,2019-07-06T17:40:54Z,4868,...,NaN,True,True,31180,31180,42,125013,1562435052,5017.0,1060
7,21737465,User,https://api.github.com/users/sindresorhus,https://github.com/sindresorhus,https://github.com/sindresorhus/awesome,https://api.github.com/repos/sindresorhus/awesome,False,2014-07-11T13:42:37Z,2019-07-06T17:41:01Z,729,...,NaN,True,False,14724,14724,18,111599,1562435052,842.0,398
8,14440270,User,https://api.github.com/users/getify,https://github.com/getify,https://github.com/getify/You-Dont-Know-JS,https://api.github.com/repos/getify/You-Dont-K...,False,2013-11-16T02:37:24Z,2019-07-06T17:00:48Z,7285,...,NaN,True,True,20641,20641,257,104485,1562435052,1487.0,160
11,1062897,Organization,https://api.github.com/users/github,https://github.com/github,https://github.com/github/gitignore,https://api.github.com/repos/github/gitignore,False,2010-11-08T20:17:14Z,2019-07-06T16:41:30Z,1956,...,NaN,False,False,42421,42421,95,86170,1562435052,3138.0,1068
13,85077558,User,https://api.github.com/users/kamranahmedse,https://github.com/kamranahmedse,https://github.com/kamranahmedse/developer-roa...,https://api.github.com/repos/kamranahmedse/dev...,False,2017-03-15T13:45:52Z,2019-07-06T17:40:33Z,23113,...,NaN,True,True,13358,13358,181,83991,1562435052,201.0,20
14,60493101,User,https://api.github.com/users/jwasham,https://github.com/jwasham,https://github.com/jwasham/coding-interview-un...,https://api.github.com/repos/jwasham/coding-in...,False,2016-06-06T02:34:12Z,2019-07-06T17:28:20Z,9499,...,NaN,True,False,23501,23501,62,79326,1562435052,1273.0,132
29,35955666,User,https://api.github.com/users/jlevy,https://github.com/jlevy,https://github.com/jlevy/the-art-of-command-line,https://api.github.com/repos/jlevy/the-art-of-...,False,2015-05-20T15:11:03Z,2019-07-06T17:39:42Z,2557,...,NaN,True,False,6528,6528,149,60235,1562435052,1185.0,153
38,14098069,User,https://api.github.com/users/justjavac,https://github.com/justjavac,https://github.com/justjavac/free-programming-...,https://api.github.com/repos/justjavac/free-pr...,False,2013-11-04T01:59:19Z,2019-07-06T16:42:32Z,848,...,NaN,True,False,16955,16955,164,51994,1562435054,885.0,
57,44571718,Organization,https://api.github.com/users/vuejs,https://github.com/vuejs,https://github.com/vuejs/awesome-vue,https://api.github.com/repos/vuejs/awesome-vue,False,2015-10-20T00:16:14Z,2019-07-06T17:40:37Z,4019,...,NaN,True,False,6475,6475,103,46726,1562435054,2748.0,1495
59,132750724,User,https://api.github.com/users/danistefanovic,https://github.com/danistefanovic,https://github.com/danistefanovic/build-your-o...,https://api.github.com/repos/danistefanovic/bu...,False,2018-05-09T12:03:18Z,2019-07-06T16:21:05Z,821,...,NaN,True,False,3131,3131,23,45917,1562435054,355.0,53


### Para alguns repositórios os contribuidores não foram extraídos. Por que?

In [62]:
contributors_faill = resultados_update.loc[resultados_csv['contributors'].isnull()]
contributors_faill

,id,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,updated_at,size,...,language,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,contributors
115,211666,Organization,https://api.github.com/users/nodejs,https://github.com/nodejs,https://github.com/nodejs/node-v0.x-archive,https://api.github.com/repos/nodejs/node-v0.x-...,False,2009-05-27T16:29:46Z,2019-07-06T12:14:12Z,144238,...,NaN,True,True,7779,7779,571,35462,1562435059,2.0,NaN
247,61929586,User,https://api.github.com/users/shengxinjing,https://github.com/shengxinjing,https://github.com/shengxinjing/programmer-job...,https://api.github.com/repos/shengxinjing/prog...,False,2016-06-25T06:09:10Z,2019-07-06T17:09:34Z,329,...,Shell,True,True,1479,1479,106,23472,1562435069,147.0,NaN
277,14098121,User,https://api.github.com/users/aosabook,https://github.com/aosabook,https://github.com/aosabook/500lines,https://api.github.com/repos/aosabook/500lines,False,2013-11-04T02:02:53Z,2019-07-06T14:43:54Z,60078,...,JavaScript,True,True,4999,4999,47,21920,1562435071,3297.0,NaN
324,3602123,Organization,https://api.github.com/users/hammerjs,https://github.com/hammerjs,https://github.com/hammerjs/hammer.js,https://api.github.com/repos/hammerjs/hammer.js,False,2012-03-02T12:58:28Z,2019-07-05T09:35:07Z,10221,...,JavaScript,True,True,2541,2541,267,20217,1562435073,1199.0,NaN
330,9603240,Organization,https://api.github.com/users/pure-css,https://github.com/pure-css,https://github.com/pure-css/pure,https://api.github.com/repos/pure-css/pure,False,2013-04-22T16:16:39Z,2019-07-06T17:08:21Z,451,...,HTML,True,True,2068,2068,19,20055,1562435075,557.0,NaN
337,28751632,User,https://api.github.com/users/0xAX,https://github.com/0xAX,https://github.com/0xAX/linux-insides,https://api.github.com/repos/0xAX/linux-insides,False,2015-01-03T18:44:57Z,2019-07-06T08:22:27Z,10236,...,Python,True,False,2128,2128,27,19862,1562435075,1644.0,NaN
370,113752225,User,https://api.github.com/users/tipsy,https://github.com/tipsy,https://github.com/tipsy/profile-summary-for-g...,https://api.github.com/repos/tipsy/profile-sum...,False,2017-12-10T13:29:15Z,2019-07-05T19:25:01Z,3200,...,Vue,True,True,388,388,14,18815,1562435077,203.0,NaN
402,60710553,Organization,https://api.github.com/users/FallibleInc,https://github.com/FallibleInc,https://github.com/FallibleInc/security-guide-...,https://api.github.com/repos/FallibleInc/secur...,False,2016-06-08T15:56:25Z,2019-07-06T13:28:47Z,5838,...,NaN,True,True,1273,1273,15,18227,1562435079,146.0,NaN
414,69798748,User,https://api.github.com/users/verekia,https://github.com/verekia,https://github.com/verekia/js-stack-from-scratch,https://api.github.com/repos/verekia/js-stack-...,False,2016-10-02T13:31:23Z,2019-07-05T22:15:50Z,1800,...,JavaScript,True,True,1833,1833,49,18012,1562435079,566.0,NaN
425,15975439,Organization,https://api.github.com/users/scrollreveal,https://github.com/scrollreveal,https://github.com/scrollreveal/scrollreveal,https://api.github.com/repos/scrollreveal/scro...,False,2014-01-16T17:37:20Z,2019-07-06T16:43:44Z,1090,...,JavaScript,True,True,2024,2024,6,17737,1562435081,931.0,NaN


Executando parte a parte da função para extrair os contributors não foi encontrado nenhum erro.

Para esses casos acredita-se que houve falha em carregar todas as informações da página antes do beatifulsoup poder extrair. Portanto, executou-se função de extração novamente, até que todos os registros conseguissem atualizar a informação de contributors. Para auxiliar na tarefa foi criada uma nova função utilizando o selenium e o chromedrive a fim de carregar a página completamente. Ainda assim para algumas execução a função não conseguia identificar o número de contributor, retornando uma informação vazia. Finalizar o processo do drive do chrome quando a função não conseguia trazer a informação apresentou uma melhora em conseguir carregar os dados. Por fim, o dado foi obtido para todos os registros, após várias execuções.

Forma de pegar o Readme de um repositório

In [ ]:
repo = g.get_repo("freeCodeCamp")
contents = repo.get_contents("README.md")
print(contents.decoded_content)